In [2]:
# pip install --upgrade git+https://github.com/hyk2202/helper.git

import pandas as pd
from helper.util import *
from helper.analysis import *
from helper.tensor import *
from helper.plot import *
from helper.timeserise import *
from helper.core import *
from helper.classification import *
from helper.re import *
from helper.regression import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hyk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
origin = my_read_data('데이터_전처리_최종.xlsx',info=False, index_col='IDX')

In [152]:
df = origin.copy()
df.columns

Index(['발생날짜', '요일', '공휴일여부', '오전오후', '시간대', '시간', '사고인지 시간', '공공/민간 구분', '날씨',
       '기온', '습도', '인적사고', '보호(방호)조치여부 - 안전방호', '보호(방호)조치여부 - 개인방호', '공종_대분류',
       '공종_중분류', '사고객체_대분류', '사고객체_중분류', '작업프로세스', '장소_대분류', '장소_중분류',
       '부위_대분류', '부위_중분류', '사고원인_주원인', '사상자수', '사고신고사유_부상', '사고신고사유_사망',
       '사고신고사유_물적', '시', '전체공사비', '전체공사비_분류', '해당공종_공사비', '구체적 사고원인',
       '사고발생후 조치사항', '공사기간_시작일', '공사기간_종료일', '공종_시작일', '공종_종료일', '재발방지대책',
       '공정률', '안전관리계획', '설계안전성검토', '작업자수', '지점 번호', '관측소', '기온.1', '강수량', '풍속',
       '풍향', '습도.1', '증기압', '이슬점온도', '현지기압', '해면기압', '일조', '전운량', '중하층운량',
       '시정', '지면온도'],
      dtype='object')

In [153]:
df.dtypes

발생날짜                 datetime64[ns]
요일                           object
공휴일여부                        object
오전오후                         object
시간대                           int64
시간                           object
사고인지 시간                      object
공공/민간 구분                     object
날씨                           object
기온                           object
습도                          float64
인적사고                         object
보호(방호)조치여부 - 안전방호            object
보호(방호)조치여부 - 개인방호            object
공종_대분류                       object
공종_중분류                       object
사고객체_대분류                     object
사고객체_중분류                     object
작업프로세스                       object
장소_대분류                       object
장소_중분류                       object
부위_대분류                       object
부위_중분류                       object
사고원인_주원인                     object
사상자수                          int64
사고신고사유_부상                    object
사고신고사유_사망                    object
사고신고사유_물적                   

In [154]:
temp = []
for i in df['인적사고']:
    if i == '기타':temp.append('분류불능')
    elif i == '없음':temp.append(None)
    else:temp.append(i)

df['인적사고'] = temp
df['인적사고'].value_counts()

넘어짐       5814
떨어짐       4321
물체에 맞음    3509
끼임        2576
분류불능      2152
부딪힘       1850
절단, 베임    1688
깔림         521
질병         381
찔림         307
화상         186
교통사고       123
감전          48
질식          18
Name: 인적사고, dtype: int64

In [155]:
temp =[]
for i in df['강수량']:
    if i>=0.1 : temp.append(1)
    else:temp.append(0)


In [156]:
# 결측치 : 0 정규작업 :1 그외: 2

temp = []
for i in df['사고인지 시간'].fillna(0):
    if i == 0 : temp.append(0)
    elif i == '정규작업 ': temp.append(1)
    else:temp.append(2)

df['사고인지 시간'] = temp
df['사고인지 시간'].value_counts()

1    18032
0     3167
2     2523
Name: 사고인지 시간, dtype: int64

In [157]:
# 결측치 : 0, 조치/해당없음:1, 비조치:2

temp = []
for i in df["보호(방호)조치여부 - 안전방호"].fillna(0):
    if i == 0 :temp.append(0)
    elif i == '비조치': temp.append(2)
    else:temp.append(1)
df["보호(방호)조치여부 - 안전방호"] =temp


df["보호(방호)조치여부 - 안전방호"].value_counts()

1    17153
0     5411
2     1158
Name: 보호(방호)조치여부 - 안전방호, dtype: int64

In [158]:
# 결측치 : 0, 조치/해당없음:1, 비조치:2

temp = []
for i in df["보호(방호)조치여부 - 개인방호"].fillna(0):
    if i == 0 :temp.append(0)
    elif i == '비조치': temp.append(2)
    else:temp.append(1)
df["보호(방호)조치여부 - 개인방호"] =temp


df["보호(방호)조치여부 - 개인방호"].value_counts()

1    17455
0     5410
2      857
Name: 보호(방호)조치여부 - 개인방호, dtype: int64

In [159]:
# 통신설비, 산업설비, 전기설비 => 기타
temp = []
for i in df['공종_대분류']:
    if i in ['통신설비','산업설비','전기설비']:temp.append('기타')
    else:temp.append(i)
df['공종_대분류'] =temp

In [160]:
# 비조치 :2 , 조치,해당없음 : 1 ,결측치 : 0
temp = []
for i in origin["작업프로세스"]:
    if i == '비조치': temp.append(2)
    elif i ==0:temp.append(1)
    else:temp.append(0)

In [161]:
#노동부고시 제2002-8호, 『화학물질 및 물리적인자의 노출기준』 중 <별표 4>의 고온의 노출기준
temp = []
for i in origin['지면온도']:
    if i <25.0: temp.append(0)
    elif i <25.9:temp.append(1)
    elif i<27.9:temp.append(2)
    elif i <30:temp.append(3)
    else:temp.append(4)
df['지면온도'] = temp

In [162]:
def daynight(x):
    if x < 6:
        return 0
    elif x <= 18:
        return 1
    else:
        return 0

In [163]:
df.columns

Index(['발생날짜', '요일', '공휴일여부', '오전오후', '시간대', '시간', '사고인지 시간', '공공/민간 구분', '날씨',
       '기온', '습도', '인적사고', '보호(방호)조치여부 - 안전방호', '보호(방호)조치여부 - 개인방호', '공종_대분류',
       '공종_중분류', '사고객체_대분류', '사고객체_중분류', '작업프로세스', '장소_대분류', '장소_중분류',
       '부위_대분류', '부위_중분류', '사고원인_주원인', '사상자수', '사고신고사유_부상', '사고신고사유_사망',
       '사고신고사유_물적', '시', '전체공사비', '전체공사비_분류', '해당공종_공사비', '구체적 사고원인',
       '사고발생후 조치사항', '공사기간_시작일', '공사기간_종료일', '공종_시작일', '공종_종료일', '재발방지대책',
       '공정률', '안전관리계획', '설계안전성검토', '작업자수', '지점 번호', '관측소', '기온.1', '강수량', '풍속',
       '풍향', '습도.1', '증기압', '이슬점온도', '현지기압', '해면기압', '일조', '전운량', '중하층운량',
       '시정', '지면온도'],
      dtype='object')

In [164]:
df['분기'] = df['발생날짜'].dt.quarter.astype('float')
df['월'] = df['발생날짜'].dt.month.astype('float')
df['주야간'] = df['시간대'].apply(daynight)
df['시간대'] = df['시간대'].astype('float')

In [165]:
df['분기']

IDX
0        2.0
1        2.0
2        2.0
3        2.0
4        2.0
        ... 
23717    3.0
23718    3.0
23719    1.0
23720    3.0
23721    3.0
Name: 분기, Length: 23722, dtype: float64

In [166]:
import pandas as pd

df['총_공사기간'] = (pd.to_datetime(df['공사기간_종료일']) - pd.to_datetime(df['공사기간_시작일'])).dt.days
df['공종공사기간'] = (pd.to_datetime(df['공종_종료일']) - pd.to_datetime(df['공종_시작일'])).dt.days
df.columns

Index(['발생날짜', '요일', '공휴일여부', '오전오후', '시간대', '시간', '사고인지 시간', '공공/민간 구분', '날씨',
       '기온', '습도', '인적사고', '보호(방호)조치여부 - 안전방호', '보호(방호)조치여부 - 개인방호', '공종_대분류',
       '공종_중분류', '사고객체_대분류', '사고객체_중분류', '작업프로세스', '장소_대분류', '장소_중분류',
       '부위_대분류', '부위_중분류', '사고원인_주원인', '사상자수', '사고신고사유_부상', '사고신고사유_사망',
       '사고신고사유_물적', '시', '전체공사비', '전체공사비_분류', '해당공종_공사비', '구체적 사고원인',
       '사고발생후 조치사항', '공사기간_시작일', '공사기간_종료일', '공종_시작일', '공종_종료일', '재발방지대책',
       '공정률', '안전관리계획', '설계안전성검토', '작업자수', '지점 번호', '관측소', '기온.1', '강수량', '풍속',
       '풍향', '습도.1', '증기압', '이슬점온도', '현지기압', '해면기압', '일조', '전운량', '중하층운량',
       '시정', '지면온도', '분기', '월', '주야간', '총_공사기간', '공종공사기간'],
      dtype='object')

In [167]:
df.drop(['발생날짜','지점 번호','관측소'],axis=1,inplace=True)

In [168]:
df['전체공사비_분류'] = df['전체공사비_분류'].map({'2,000만 ~ 5억원 미만':2, '2,000만 미만':1, '50억원 이상':4, '5억 ~ 50억원 미만':3, '분류불능':0})
df['전체공사비_분류'].value_counts()

4    15759
2     3961
3     3654
1      228
0      120
Name: 전체공사비_분류, dtype: int64

In [169]:
temp = []
for i in df['작업자수']:
    try:
        a = int(i[:3])
        if a == 100: temp.append(4)
        elif a == 300: temp.append(5)
        elif a == 500: temp.append(6)

    except:
        a = int(i[:2])
        if a ==19:temp.append(0)
        elif a == 20 :temp.append(1)
        elif a==50:temp.append(2)
df['작업자수'] = temp


In [170]:
temp = []
for i in origin['공정률']:
    try:
        if '미만' in i : temp.append(0)
        elif '이상' in i: temp.append(9)
        else:
            temp.append(int(i[:1]))
    except:
        
        temp.append(0)

df['공정률'] = temp

In [171]:
df.dtypes

    
fields=[]
for f in df.columns:
    if df[f].dtypes not in [
        "int",
        "int32",
        "int64",
        "float",
        "float32",
        "float64",
    ]:
        fields.append(f)

In [172]:
cate = df[fields]
df2 = df.drop(fields,axis=1)

In [173]:
df2 = my_replace_missing_value(df2,'constant',0)
cate = my_replace_missing_value(cate,'constant','기타')

In [174]:
fields.remove('공사기간_시작일')
fields.remove('공사기간_종료일')
fields.remove('공종_시작일')
fields.remove('공종_종료일')
fields.remove('구체적 사고원인')
fields.remove('사고발생후 조치사항')
fields.remove('재발방지대책')

In [175]:
drop_fields = ['시간대','습도']

In [176]:
df2.columns

Index(['시간대', '사고인지 시간', '습도', '보호(방호)조치여부 - 안전방호', '보호(방호)조치여부 - 개인방호',
       '사상자수', '전체공사비_분류', '공정률', '작업자수', '기온.1', '강수량', '풍속', '풍향', '습도.1',
       '증기압', '이슬점온도', '현지기압', '해면기압', '일조', '전운량', '중하층운량', '시정', '지면온도',
       '분기', '월', '주야간', '총_공사기간', '공종공사기간'],
      dtype='object')

In [177]:
cate = cate[fields]

In [178]:
drop_cols = ['기온', '습도', '구체적 사고원인', '사고발생후 조치사항', '재발방지대책', '공사기간_시작일', '공사기간_종료일', '공종_시작일',
             '공종_종료일', '관측소', '시간', '발생날짜', '지점 번호', '전체공사비_분류']

In [179]:
drop_cols = [x for x in drop_cols if x in df2.columns]
drop_cols

['습도', '전체공사비_분류']

In [180]:
df2 = df2.drop(drop_cols,axis=1)
df2[fields]=cate
df2.dtypes
# fields.remove('인적사고')


시간대                  float64
사고인지 시간              float64
보호(방호)조치여부 - 안전방호    float64
보호(방호)조치여부 - 개인방호    float64
사상자수                 float64
공정률                  float64
작업자수                 float64
기온.1                 float64
강수량                  float64
풍속                   float64
풍향                   float64
습도.1                 float64
증기압                  float64
이슬점온도                float64
현지기압                 float64
해면기압                 float64
일조                   float64
전운량                  float64
중하층운량                float64
시정                   float64
지면온도                 float64
분기                   float64
월                    float64
주야간                  float64
총_공사기간               float64
공종공사기간               float64
요일                    object
공휴일여부                 object
오전오후                  object
시간                    object
공공/민간 구분              object
날씨                    object
기온                    object
인적사고                  object
공종_대분류        

In [181]:
df2 = df2.dropna()

In [182]:
df2 = df2[df2['인적사고'] != '0'].dropna()

In [183]:
fields = [x for x in fields if x in df2.columns]
fields

['요일',
 '공휴일여부',
 '오전오후',
 '시간',
 '공공/민간 구분',
 '날씨',
 '기온',
 '인적사고',
 '공종_대분류',
 '공종_중분류',
 '사고객체_대분류',
 '사고객체_중분류',
 '작업프로세스',
 '장소_대분류',
 '장소_중분류',
 '부위_대분류',
 '부위_중분류',
 '사고원인_주원인',
 '사고신고사유_부상',
 '사고신고사유_사망',
 '사고신고사유_물적',
 '시',
 '전체공사비',
 '해당공종_공사비',
 '안전관리계획',
 '설계안전성검토']

In [184]:
df2.columns

Index(['시간대', '사고인지 시간', '보호(방호)조치여부 - 안전방호', '보호(방호)조치여부 - 개인방호', '사상자수',
       '공정률', '작업자수', '기온.1', '강수량', '풍속', '풍향', '습도.1', '증기압', '이슬점온도',
       '현지기압', '해면기압', '일조', '전운량', '중하층운량', '시정', '지면온도', '분기', '월', '주야간',
       '총_공사기간', '공종공사기간', '요일', '공휴일여부', '오전오후', '시간', '공공/민간 구분', '날씨', '기온',
       '인적사고', '공종_대분류', '공종_중분류', '사고객체_대분류', '사고객체_중분류', '작업프로세스', '장소_대분류',
       '장소_중분류', '부위_대분류', '부위_중분류', '사고원인_주원인', '사고신고사유_부상', '사고신고사유_사망',
       '사고신고사유_물적', '시', '전체공사비', '해당공종_공사비', '안전관리계획', '설계안전성검토'],
      dtype='object')

In [185]:
df2['설계안전성검토'].value_counts()

비대상    15212
대상      8313
기타         1
Name: 설계안전성검토, dtype: int64

In [186]:
df2.dtypes

시간대                  float64
사고인지 시간              float64
보호(방호)조치여부 - 안전방호    float64
보호(방호)조치여부 - 개인방호    float64
사상자수                 float64
공정률                  float64
작업자수                 float64
기온.1                 float64
강수량                  float64
풍속                   float64
풍향                   float64
습도.1                 float64
증기압                  float64
이슬점온도                float64
현지기압                 float64
해면기압                 float64
일조                   float64
전운량                  float64
중하층운량                float64
시정                   float64
지면온도                 float64
분기                   float64
월                    float64
주야간                  float64
총_공사기간               float64
공종공사기간               float64
요일                    object
공휴일여부                 object
오전오후                  object
시간                    object
공공/민간 구분              object
날씨                    object
기온                    object
인적사고                  object
공종_대분류        

In [187]:
df2.columns

Index(['시간대', '사고인지 시간', '보호(방호)조치여부 - 안전방호', '보호(방호)조치여부 - 개인방호', '사상자수',
       '공정률', '작업자수', '기온.1', '강수량', '풍속', '풍향', '습도.1', '증기압', '이슬점온도',
       '현지기압', '해면기압', '일조', '전운량', '중하층운량', '시정', '지면온도', '분기', '월', '주야간',
       '총_공사기간', '공종공사기간', '요일', '공휴일여부', '오전오후', '시간', '공공/민간 구분', '날씨', '기온',
       '인적사고', '공종_대분류', '공종_중분류', '사고객체_대분류', '사고객체_중분류', '작업프로세스', '장소_대분류',
       '장소_중분류', '부위_대분류', '부위_중분류', '사고원인_주원인', '사고신고사유_부상', '사고신고사유_사망',
       '사고신고사유_물적', '시', '전체공사비', '해당공종_공사비', '안전관리계획', '설계안전성검토'],
      dtype='object')

In [188]:
for i in df2.columns:
    if df2[i].dtypes not in ['float64','float32','float']:
        print(i,df2[i].nunique())
    

요일 7
공휴일여부 2
오전오후 2
시간 792
공공/민간 구분 3
날씨 6
기온 591
인적사고 15
공종_대분류 4
공종_중분류 39
사고객체_대분류 9
사고객체_중분류 54
작업프로세스 34
장소_대분류 37
장소_중분류 6
부위_대분류 54
부위_중분류 10
사고원인_주원인 29
사고신고사유_부상 2
사고신고사유_사망 2
사고신고사유_물적 2
시 18
전체공사비 18
해당공종_공사비 19
안전관리계획 4
설계안전성검토 3


In [189]:


x = my_dummies(df2.drop(['인적사고','기온','시간'],axis=1),'요일', '공휴일여부', '오전오후',  '공공/민간 구분', '날씨',   '공종_대분류',
       '공종_중분류', '사고객체_대분류', '사고객체_중분류', '작업프로세스', '장소_대분류', '장소_중분류',
       '부위_대분류', '부위_중분류', '사고원인_주원인', '사고신고사유_부상', '사고신고사유_사망', '사고신고사유_물적',
       '시', '전체공사비', '해당공종_공사비', '안전관리계획', '설계안전성검토')
y = my_labelling(df2[['인적사고']],'인적사고')


In [190]:
x.shape

(23526, 367)

In [191]:
x['인적사고'] = y

In [193]:
x_train,x_test,y_train,y_test = my_train_test_split(x,'인적사고')
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((18820, 367), (4706, 367), (18820,), (4706,))

In [194]:
# params = {
#     "n_estimators": [10, 20, 50, 100],
#     "criterion": ["squared_error", "absolute_error", "friedman_mse", "poisson"],
#     "max_features": ["sqrt", "log2"],
#     "max_depth": [10, 20, 50, None],
# }

xgb - y 갯수 줄이기 전

{'subsample': 1, 'reg_lambda': 7, 'reg_alpha': 1, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 6, 'learning_rate': 0.1, 'gamma': 2, 'colsample_bytree': 0.7}

[분류분석 성능평가]


| | 훈련데이터 |   검증데이터 | 설명                                                      |
|--|--|--|--|
| 정확도(Accuracy)  |     0.541919 |     0.488936 | 예측 결과(TN,FP,TP,TN)가 실제 결과(TP,TN)와 일치하는 정도 |
| 정밀도(Precision) |     0.655207 |     0.420337 | 양성으로 예측한 결과(TP,FP) 중 실제 양성(TP)인 비율       |
| 재현율(Recall)    |     0.541919 |     0.335646 | 실제 양성(TP,FN) 중 양성(TP)으로 예측한 비율              |
| F1 Score          |     0.409932 |     0.357504 | 정밀도와 재현율의 조화평균                                |
| AUC(ovo)          |     0.917224 |     0.868432 | One vs One에 대한 AUC로, 1에 가까울수록 좋은 모델         |
| AUC(ovr)          |     0.913331 |     0.860516 | One vs Rest에 대한 AUC로, 1에 가까울수록 좋은 모델        |



[Best Params]
{'subsample': 1, 'reg_lambda': 7, 'reg_alpha': 1, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 6, 'learning_rate': 0.1, 'gamma': 2, 'colsample_bytree': 0.7}



|                   |   훈련데이터 |   검증데이터 | 설명                                                      |
|--|--|--|--|
| 정확도(Accuracy)  |     0.545696 |     0.509137 | 예측 결과(TN,FP,TP,TN)가 실제 결과(TP,TN)와 일치하는 정도 |
| 정밀도(Precision) |     0.675546 |     0.469469 | 양성으로 예측한 결과(TP,FP) 중 실제 양성(TP)인 비율       |
| 재현율(Recall)    |     0.545696 |     0.361035 | 실제 양성(TP,FN) 중 양성(TP)으로 예측한 비율              |
| F1 Score          |     0.416669 |     0.381702 | 정밀도와 재현율의 조화평균                                |
| AUC(ovo)          |     0.894882 |     0.868483 | One vs One에 대한 AUC로, 1에 가까울수록 좋은 모델         |
| AUC(ovr)          |     0.895783 |     0.861934 | One vs Rest에 대한 AUC로, 1에 가까울수록 좋은 모델        |

In [195]:
my_xgb_classification(x_train,y_train,x_test,y_test, pr=False,roc=False,hist=False,learning_curve=False)